In [1]:
import gsd
import hoomd
import gsd.hoomd
import numpy as np
from cmeutils import gsd_utils

In [2]:
gsd_file = gsd.hoomd.open("100_10mer10f_0.0005dt_7kT_large.gsd")
snap = gsd_file[65000]
positions = gsd_utils.get_type_position("F", snap=snap)
particle_types = gsd_utils.get_all_types(snap=snap)
delete_chains = gsd_utils.snap_delete_types(snap, 'A')
flakes = gsd_utils.get_molecule_cluster(snap=delete_chains)

In [3]:
def compute_flake_COMs(positions, flake_labels):
    flake_ids = np.unique(flake_labels)
    coms = [positions[flake_labels == f].mean(axis=0) for f in flake_ids]
    return np.array(coms), flake_ids

def find_neighbors(coms, flake_ids, cutoff):
    neighbors = {f: [] for f in flake_ids}
    for i, f in enumerate(flake_ids):
        for j, g in enumerate(flake_ids):
            if i != j and np.linalg.norm(coms[i] - coms[j]) <= cutoff:
                neighbors[f].append(g)
    return neighbors

In [4]:
cutoff = 2.0
coms, flake_ids = compute_flake_COMs(positions, flakes)
neighbors = find_neighbors(coms, flake_ids, cutoff)

In [5]:
for flake_id in range(10):
    neighbor_ids = neighbors.get(flake_id, [])
    print(f"[{flake_id}] Neighbors: {neighbor_ids}")

[0] Neighbors: [6]
[1] Neighbors: [7]
[2] Neighbors: []
[3] Neighbors: []
[4] Neighbors: []
[5] Neighbors: []
[6] Neighbors: [0]
[7] Neighbors: [1]
[8] Neighbors: [9]
[9] Neighbors: [8]


In [6]:
#need to keep working on this, not sure if measuring flake cluster correctly 
import networkx as nx
G = nx.Graph()
G.add_nodes_from(flake_ids)

for flake, nbrs in neighbors.items():
    for nbr in nbrs:
        G.add_edge(flake, nbr)
        
flake_clusters = list(nx.connected_components(G))
for i, cluster in enumerate(flake_clusters):
    print(f"Flake cluster {i}: {sorted(cluster)}")

Flake cluster 0: [0, 6]
Flake cluster 1: [1, 7]
Flake cluster 2: [2]
Flake cluster 3: [3]
Flake cluster 4: [4]
Flake cluster 5: [5]
Flake cluster 6: [8, 9]
